This notebook just for try to train in dev data before deep dive in training data

this is important:
https://groups.google.com/g/sigarab/c/8e3Os4Q0Aac

# Trying to learn how to do NLP and LLms Predictions
ref :
https://towardsdatascience.com/machine-learning-word-embedding-and-predicting-e603254e4d7b

In [ ]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [ ]:
with open('dev.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

In [1]:
import pandas as pd

df = pd.DataFrame(data)

df.info()

"""
This is base frameworks used:
 - Electra (Clark et al., 2020) 
 - BERT (Devlin et al., 2019),


in our dataframe Embedding:
- AraELECTRA (Antoun et al., 2021)    -> electra 
- AraBERTv2 (Antoun et al., 2020)     ->  bertseg
- camelBERT-MSA (Inoue et al., 2021)  -> bertmsa
"""

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3921 entries, 0 to 3920
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       3921 non-null   object
 1   word     3921 non-null   object
 2   pos      3636 non-null   object
 3   gloss    3921 non-null   object
 4   electra  3921 non-null   object
 5   bertseg  3921 non-null   object
 6   bertmsa  3921 non-null   object
dtypes: object(7)
memory usage: 214.6+ KB


'\nThis is base frameworks used:\n - Electra (Clark et al., 2020) \n - BERT (Devlin et al., 2019),\n\n\nin our dataframe Embedding:\n- AraELECTRA (Antoun et al., 2021)    -> electra \n- AraBERTv2 (Antoun et al., 2020)     ->  bertseg\n- camelBERT-MSA (Inoue et al., 2021)  -> bertmsa\n'

In [2]:
for i in dev[:10]:
    print(i['id'], i['word'], i["pos"], i["gloss"])

ar.962714 أَكْمَدَ V غَمَّ وَأَمْرَضَ القَلْبَ
ar.994971 ب None حرف جرّ زائد في التوكيد بالنفس والعين .
ar.989034 ذَكَا V ذَكَا الشَّاةَ ونحوَها: ذَبَحَها
ar.994539 وَرَع V وَرَعَ فلانٌ: جَبُنَ.
ar.992756 قَلَص V قَلَصَ الظِّلُّ عنِّي: انْقَبَض ونَقَصَ.
ar.993508 أَمْرَضَ V أَمْرَضَ فلانٌ: قَارَبَ الصَّوابَ في الرَّأْي وإِن لم يُصِبْ كلَّ الصَّواب.
ar.977791 عليق N : ما يُقدَّم للدَّابَّة من طعام "عليقٌ من شعير/ حشيش- عليقٌ يابس".
ar.977267 تطوع V تبرَّع به، وزاد على ما يجب عليه
ar.968835 قَصْر اَلرِّئَاسَةِ None مقرّ إقامة رئيس الدولة.
ar.978831 مقدم N من يلْتَمس شيئًا .


In [ ]:
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf

# Load pre-trained BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

# Example input
word = "example"
pos = "noun"
gloss = "a thing characteristic of its kind or illustrating a general rule."

# Tokenize inputs
word_tokens = tokenizer.encode(word, add_special_tokens=False, max_length=512, truncation=True)
pos_token = tokenizer.encode(pos, add_special_tokens=False, max_length=512, truncation=True)
gloss_tokens = tokenizer.encode(gloss, add_special_tokens=False, max_length=512, truncation=True)

In [3]:
# Combine token IDs and add special tokens
# input_ids = tokenizer.build_inputs_with_special_tokens(word_tokens, pos_token, gloss_tokens)
input_ids = [tokenizer.cls_token_id] + word_tokens + [tokenizer.sep_token_id] + pos_token + [tokenizer.sep_token_id] + gloss_tokens + [tokenizer.sep_token_id]

print(input_ids)
# Convert to tensor
input_ids = tf.convert_to_tensor(input_ids)

print(input_ids)
# Get BERT embeddings
outputs = bert_model(input_ids)
# word_embedding = outputs[0][:, 0, :]  # Assuming you want to use the [CLS] token as the word embedding

# Further process or concatenate embeddings as needed
# ...

# Now you can use word_embedding as the input to your model

[101, 2742, 102, 15156, 102, 1037, 2518, 8281, 1997, 2049, 2785, 2030, 28252, 1037, 2236, 3627, 1012, 102]
tf.Tensor(
[  101  2742   102 15156   102  1037  2518  8281  1997  2049  2785  2030
 28252  1037  2236  3627  1012   102], shape=(18,), dtype=int32)
---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
Cell In[62], line 11
      9 print(input_ids)
     10 # Get BERT embeddings
---> 11 outputs = bert_model(input_ids)
     12 # word_embedding = outputs[0][:, 0, :]  # Assuming you want to use the [CLS] token as the word embedding
     13 
     14 # Further process or concatenate embeddings as needed
     15 # ...
     16 
     17 # Now you can use word_embedding as the input to your model

File ~/.local/lib/python3.11/site-packages/keras/src/utils/traceback_utils.py:70, in filter_traceback.<locals>.error_handler(*args, **kwargs)
     67     filtered_tb = _process_traceback_frames(e.__tr

In [4]:
X = df[['word', 'pos', 'gloss']]
y = df[['electra', 'bertseg', 'bertmsa']]

# Preprocess the data if necessary
# For example, you might need to handle missing values or tokenize the text


# Tokenize the glosses and words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X["gloss"])

# # Convert text to sequences
# X = tokenizer.texts_to_sequences(glosses)
# y = tokenizer.texts_to_sequences(words)

# Pad sequences to ensure uniform length
max_len = max(len(seq) for seq in X)
X_padded = pad_sequences(X, maxlen=max_len)
y_padded = pad_sequences(y, maxlen=max_len)

# Prepare the data for training
X_train, X_val, y_train, y_val = train_test_split(X_padded, y_padded, test_size=0.2, random_state=42)



# Step 2: Model Building
vocab_size = 10000  # Define your vocabulary size
embedding_dim = 50  # Define the dimensionality of your word embeddings

model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=3),  # Assuming 3 input features
    LSTM(units=64),
    Dense(units=3)  # Output dimension should match the number of output columns (3 in this case)
])

model.compile(optimizer='adam', loss='mse')  # Use mean squared error as the loss function

model.summary()
# Step 3: Training
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)

# Step 4: Evaluation
loss = model.evaluate(X_val, y_val)

# Step 5: Inference
# Use the trained model to generate output predictions for new input data

# Step 6: Model Deployment
# Deploy the trained model if necessary

---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
Cell In[22], line 18
     12 # # Convert text to sequences
     13 # X = tokenizer.texts_to_sequences(glosses)
     14 # y = tokenizer.texts_to_sequences(words)
     15 
     16 # Pad sequences to ensure uniform length
     17 max_len = max(len(seq) for seq in X)
---> 18 X_padded = pad_sequences(X, maxlen=max_len)
     19 y_padded = pad_sequences(y, maxlen=max_len)
     21 # Prepare the data for training

File ~/.local/lib/python3.11/site-packages/keras/src/utils/data_utils.py:1132, in pad_sequences(sequences, maxlen, dtype, padding, truncating, value)
   1129     raise ValueError(f'Truncating type "{truncating}" not understood')
   1131 # check `trunc` has expected shape
-> 1132 trunc = np.asarray(trunc, dtype=dtype)
   1133 if trunc.shape[1:] != sample_shape:
   1134     raise ValueError(
   1135         f"Shape of sample {trunc.shape